In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib

In [ ]:
save_root = "./save_asap"
save_root = pathlib.Path(save_root)
if not save_root.exists():
    raise FileExistsError(save_root)
elif not save_root.is_dir():
    raise FileExistsError(save_root)

scatter_2d_path = save_root / "scatter_2d.png"
if not scatter_2d_path.exists():
    raise FileExistsError(scatter_2d_path)
elif not scatter_2d_path.is_file():
    raise FileExistsError(scatter_2d_path)

scatter_3d_path = save_root / "scatter_3d.png"
if not scatter_3d_path.exists():
    raise FileExistsError(scatter_3d_path)
elif not scatter_3d_path.is_file():
    raise FileExistsError(scatter_3d_path)

pos_csv_path = save_root / "pos.csv"
if not pos_csv_path.exists():
    raise FileExistsError(pos_csv_path)
elif not pos_csv_path.is_file():
    raise FileExistsError(pos_csv_path)
pos_csv = pd.read_csv(pos_csv_path)

neg_csv_path = save_root / "neg.csv"
if not neg_csv_path.exists():
    raise FileExistsError(neg_csv_path)
elif not neg_csv_path.is_file():
    raise FileExistsError(neg_csv_path)
neg_csv = pd.read_csv(neg_csv_path)

npz_root = save_root / "npz"
if not npz_root.exists():
    raise FileExistsError(npz_root)
elif not npz_root.is_dir():
    raise FileExistsError(npz_root)
pos_npz_paths = list(npz_root.glob("*pos*"))
neg_npz_paths = list(npz_root.glob("*neg*"))

num_pos_npz = len(pos_npz_paths)
num_neg_npz = len(neg_npz_paths)

print(f"# of pos_*.npz files: {num_pos_npz}")
print(f"# of neg_*.npz files: {num_neg_npz}")

fig, ax = plt.subplots(1,2, figsize=(20, 10))
ax[0].axis("off")
ax[0].imshow(plt.imread(scatter_2d_path))
ax[1].axis("off")
ax[1].imshow(plt.imread(scatter_3d_path))
plt.show()
plt.clf()

In [ ]:
histogram_min, histogram_max = (0, 0.05)
timewarping_min, timewarping_max = (0, 10)

print(histogram_min, histogram_max)

In [ ]:
selected_pos = pos_csv.loc[pos_csv["Histogram distance"] < histogram_max]
selected_pos